In [1]:
# Programação simbólica
# import sympy as sp
# from sympy.plotting import plot, plot3d
 
# Exibir símbolos nas células
# from google.colab.output._publish import javascript
# url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/3.1.2/latest.js?config=default"
 
# javascript(url=url)
from IPython.display import Math

# Manipulação dos dados
import numpy as np
import pandas as pd
 
# Gráficos
import matplotlib as mpl
import matplotlib.pyplot as plt
 
import seaborn as sns
sns.set_theme()

np.random.seed(42)

In [2]:
# Ajustes nas configurações padrões dos plots
mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['font.size'] = 14
mpl.rcParams['legend.fontsize'] = 14

# Processos Gaussianos para Regressão

Baseado em: https://gpflow.readthedocs.io/en/develop/notebooks/basics/regression.html

## Preparação dos dados

Carregando dataset

In [3]:
dataset = pd.read_csv('housesToRentProcessed.csv')

dataset

,São Paulo,Porto Alegre,Rio de Janeiro,Campinas,Belo Horizonte,area,rooms,bathroom,parking spaces,floor,isHouse,animal,furniture,y con + alu
0,1,0,0,0,0,70,2,1,1,7,0,1,1,5365
1,1,0,0,0,0,320,4,4,0,20,0,1,0,6160
2,0,1,0,0,0,80,1,1,1,6,0,1,0,3800
3,0,1,0,0,0,51,2,1,0,2,0,1,0,1382
4,1,0,0,0,0,25,1,1,0,1,0,0,0,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,0,1,0,0,0,63,2,1,1,5,0,0,1,1880
10688,1,0,0,0,0,285,4,4,4,17,0,1,0,18100
10689,0,0,1,0,0,70,3,3,0,8,0,0,1,6980
10690,0,0,1,0,0,120,2,2,2,8,0,1,1,13585


Definindo `X` e `y`

In [4]:
X = dataset[[
    'São Paulo', 'Porto Alegre', 'Rio de Janeiro', 'Campinas', 'Belo Horizonte', 
    'area',
    'rooms', 'bathroom', 'parking spaces', 'floor', 'isHouse', 'animal', 'furniture',
]].values
Y = dataset[['y con + alu']].values

#N, D = X.shape

Normalização dos dados abaixo foi simplificada, para somente testar o modelo

In [5]:
X = X/X.max()
Y = Y/Y.max()

## Definição do modelo

In [6]:
import gpflow
import tensorflow as tf
from gpflow.utilities import print_summary

from trabalho.modelos.gp import GP

# class GP:
#     def __init__(self, kernel):
#         self.kernel = kernel
#         self.modelo = None

#     def fit(self, X, Y):
#         self.modelo = gpflow.models.GPR(data=(X, Y), kernel=self.kernel, mean_function=None)
        
#         print_summary(self.kernel)
#         print_summary(self.modelo)

#     def predict(self, X):
#         return self.modelo.predict_f(X)

gp = GP(
    kernel=gpflow.kernels.Matern52(),
)

## Treinamento do modelo

In [7]:
gp.fit(X[:50], Y[:50])

In [8]:
from sklearn.metrics import mean_squared_error

rmse = lambda a, b: mean_squared_error(a, b, squared=False)
a, b = gp.predict(X[:50])

rmse(a, Y[:50])

0.003044047285746297